A sentiment classifier that outperforms the current top HuggingFace leaderboard on the `rotten_tomatoes` sentiment dataset.

In [7]:
import sys
import os

notebook_dir = os.path.abspath('')
parent_dir = os.path.dirname(notebook_dir)
sys.path.insert(0, parent_dir)

In [8]:
from dataset import Dataset
from model import ClassificationModel
import json

In [9]:
# Load the config file
type = "sentiment_imdb"
# type = "sentiment_rottentomatoes"

with open (f'configs/{type}.json') as f:
    config = json.load(f)

In [10]:
# Create the test and validation datasets
config_dataset = config['datasets'][0]
dataset_train = Dataset(type, config_dataset, save=True, split='train')
dataset_test = Dataset(type, config_dataset, save=True, split='test')

Found cached dataset rotten_tomatoes (/Users/dmvaldman/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Loading cached processed dataset at /Users/dmvaldman/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-0927eda287c4ebcf.arrow


In [ ]:
# Fit and test the linear model
config_model = config['model']
model = ClassificationModel(config_model)

result_train = model.fit(dataset_train)
print('Training Results', result_train)

result_test = model.test(dataset_test)
print('Validation Results:', result_test)

Training Results {'accuracy': 0.9336, 'f1': 0.9335914300006051}
Validation Results: {'accuracy': 0.92996, 'f1': 0.9299540196423075}


In [ ]:
path = f'models/{type}/model.npz'
model.save(path)